# **🛠️ Step 1: Library Imports & Initial Setup**

In [ ]:
# 📚 Essential Libraries
import numpy as np
import pandas as pd

# 📊 Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns

# 🔍 Statistical Libraries
from scipy.stats import norm, skew
from scipy import stats

# ⚠️ Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# 🎨 Set visualization style
sns.set(style="whitegrid")

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation

print("✅ Libraries Imported Successfully!")


# **📥 Step 2: Data Loading & Initial Exploration**

In [ ]:
# 📥 Load datasets
train = pd.read_csv("/kaggle/input/home-data-for-ml-course/train.csv")
test = pd.read_csv("/kaggle/input/home-data-for-ml-course/test.csv")

# 🔍 Initial exploration
print("✅ Data Loaded Successfully!")

# 🔢 Shape of datasets
print(f"📌 Training Data Shape: {train.shape}")
print(f"📌 Test Data Shape: {test.shape}")

# 🔎 First look at training data
print("\n🔹 First 5 rows of Training Data:")
display(train.head())

# 🔎 First look at test data
print("\n🔹 First 5 rows of Test Data:")
display(test.head())


# **🧩 Step 3: Understanding Data**

In [ ]:
# 🔬 Data Types and Info
print("\n🔹 Training Data Information:")
train.info()

print("\n🔹 Test Data Information:")
test.info()

# 📈 Basic Statistics
print("\n🔹 Basic Statistics of Numerical Features (Train Data):")
display(train.describe())

print("\n🔹 Basic Statistics of Numerical Features (Test Data):")
display(test.describe())

# 📑 Number of Unique Values for categorical features
print("\n🔹 Unique Values in Categorical Features (Training Data):")
categorical_cols = train.select_dtypes(include=['object']).columns
for col in categorical_cols:
    print(f"{col}: {train[col].nunique()} unique values")


# **🚩 Step 4: Missing Values Analysis**

In [ ]:
# 🔍 Check missing values for Training data
missing_train = train.isnull().sum()
missing_train = missing_train[missing_train > 0].sort_values(ascending=False)

print("\n🔹 Missing Values in Training Data:")
display(missing_train)

# 🔍 Check missing values for Test data
missing_test = test.isnull().sum()
missing_test = missing_test[missing_test > 0].sort_values(ascending=False)

print("\n🔹 Missing Values in Test Data:")
display(missing_test)

# 📊 Visualizing missing values (train data)
plt.figure(figsize=(12, 8))
sns.barplot(y=missing_train.index, x=missing_train.values, palette="viridis")
plt.title('Missing Values in Training Data', fontsize=16)
plt.xlabel('Number of Missing Values')
plt.ylabel('Features')
plt.show()

# 📊 Visualizing missing values (test data)
plt.figure(figsize=(12, 8))
sns.barplot(y=missing_test.index, x=missing_test.values, palette="magma")
plt.title('Missing Values in Test Data', fontsize=16)
plt.xlabel('Number of Missing Values')
plt.ylabel('Features')
plt.show()


# **📊 Step 5: Distribution Analysis (Target & Key Features)**

## 🎯 Target Variable: SalePrice Analysis

In [ ]:
# 🎯 Distribution of SalePrice
plt.figure(figsize=(10,6))
sns.histplot(train['SalePrice'], kde=True, color='skyblue')
plt.title('Distribution of SalePrice', fontsize=16)
plt.xlabel('SalePrice')
plt.ylabel('Frequency')
plt.show()

# 🧮 Skewness & Kurtosis
print(f"Skewness: {train['SalePrice'].skew():.3f}")
print(f"Kurtosis: {train['SalePrice'].kurt():.3f}")


## 🏡 Important Numerical Features Analysis

In [ ]:
num_features = ['GrLivArea', 'TotalBsmtSF', '1stFlrSF', 'GarageArea', 'LotArea']

for feature in num_features:
    plt.figure(figsize=(8,5))
    sns.scatterplot(x=train[feature], y=train['SalePrice'], alpha=0.5)
    plt.title(f'SalePrice vs {feature}', fontsize=16)
    plt.xlabel(feature)
    plt.ylabel('SalePrice')
    plt.show()


## 🔖 Important Categorical Features Analysis

In [ ]:
cat_features = ['OverallQual', 'Neighborhood', 'GarageCars']

for feature in cat_features:
    plt.figure(figsize=(14,6))
    sns.boxplot(x=feature, y='SalePrice', data=train, palette="coolwarm")
    plt.title(f'SalePrice distribution by {feature}', fontsize=16)
    plt.xticks(rotation=45)
    plt.show()


# **🔗 Step 6: Feature Relationships & Correlation Analysis**

In [ ]:
# 🔧 Select numeric columns for correlation analysis
numeric_features = train.select_dtypes(include=['int64', 'float64']).columns
corr_matrix = train[numeric_features].corr()

# 🔍 Top 15 correlated numeric features with SalePrice
top_corr_features = corr_matrix['SalePrice'].abs().sort_values(ascending=False).head(15).index

# 📌 Heatmap
plt.figure(figsize=(12,10))
sns.heatmap(train[top_corr_features].corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('🔝 Top Correlated Features Heatmap', fontsize=16)
plt.show()


# **🎨 Step 7: Advanced Visualizations (Pairplot)**

In [ ]:
sns.pairplot(train[top_corr_features[:6]], corner=True)
plt.suptitle('🔗 Pairplot of Top Correlated Features', fontsize=16, y=1.02)
plt.show()

In [ ]:
y = train["SalePrice"]
train_id = train["Id"]
test_id = test["Id"]

# 🧹 Preprocessing function
def preprocess(df):
    df = df.copy()
    for col in df.select_dtypes(include="object"):
        df[col] = df[col].fillna("None")
        df[col] = LabelEncoder().fit_transform(df[col])
    for col in df.select_dtypes(include=["int64", "float64"]):
        df[col] = df[col].fillna(df[col].median())
    return df

# 🎯 Feature separation
X = preprocess(train.drop(["Id", "SalePrice"], axis=1))
X_test = preprocess(test.drop("Id", axis=1))

# 🔁 K-Fold Cross Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
oof = np.zeros(len(X))
preds = np.zeros(len(X_test))

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    model = lgb.LGBMRegressor(n_estimators=1000, learning_rate=0.01, random_state=42)
    
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        callbacks=[
            early_stopping(stopping_rounds=100),
            log_evaluation(period=100)
        ]
    )
    
    oof[val_idx] = model.predict(X_val)
    preds += model.predict(X_test) / kf.n_splits

# 🧮 CV RMSE
rmse = mean_squared_error(y, oof, squared=False)
print(f"✅ CV RMSE: {rmse:.4f}")

# 💾 Submission file
submission = pd.DataFrame({
    "Id": test_id,
    "SalePrice": preds
})
submission.to_csv("submission.csv", index=False)
print("🚀 submission.csv has been created.")
